In [1]:
%reset -f

In [2]:
user = "HHegde"
db = f"/Users/{user}/.data/oaklib/phenio.db"

In [3]:
%reload_ext sql
%sql sqlite:///{db}

In [4]:
%sql SELECT * FROM term_association LIMIT 2;

 * sqlite:////Users/HHegde/.data/oaklib/phenio.db
Done.


id,subject,predicate,object,evidence_type,publication,source
uuid:70269c5a-42a9-11ee-be37-31ef105c25ea,MONDO:0023659,biolink:has_phenotype,HP:0011097,ECO:0000269,PMID:31675180,infores:hpo-annotations
uuid:70269c5b-42a9-11ee-be37-31ef105c25ea,MONDO:0023659,biolink:has_phenotype,HP:0002187,ECO:0000269,PMID:31675180,infores:hpo-annotations


In [5]:
import sqlite3
import pandas as pd
from semsimian import Semsimian
from collections import Counter

In [6]:
conn = sqlite3.connect(db)
res = conn.execute("SELECT name FROM sqlite_master WHERE type='table';")
# tables = res.fetchall()

# tables

In [7]:
df_term_association = pd.read_sql_query("SELECT * FROM term_association", conn)
df_term_association.head()

,id,subject,predicate,object,evidence_type,publication,source
0,uuid:70269c5a-42a9-11ee-be37-31ef105c25ea,MONDO:0023659,biolink:has_phenotype,HP:0011097,ECO:0000269,PMID:31675180,infores:hpo-annotations
1,uuid:70269c5b-42a9-11ee-be37-31ef105c25ea,MONDO:0023659,biolink:has_phenotype,HP:0002187,ECO:0000269,PMID:31675180,infores:hpo-annotations
2,uuid:70269c5c-42a9-11ee-be37-31ef105c25ea,MONDO:0023659,biolink:has_phenotype,HP:0001518,ECO:0000269,PMID:31675180,infores:hpo-annotations
3,uuid:70269c5d-42a9-11ee-be37-31ef105c25ea,MONDO:0023659,biolink:has_phenotype,HP:0032792,ECO:0000269,PMID:31675180,infores:hpo-annotations
4,uuid:70269c5e-42a9-11ee-be37-31ef105c25ea,MONDO:0023659,biolink:has_phenotype,HP:0011451,ECO:0000269,PMID:31675180,infores:hpo-annotations


df_term_association['subject'].str.split(":").str[0].unique()

In [8]:
df_term_association['object'].str.split(":").str[0].unique()

array(['HP', 'WBPhenotype', 'MP', 'XPO', 'ZP'], dtype=object)

In [11]:
# MGI:1261425 => drosha, ribonuclease type III
condition_1 = df_term_association['subject'] == "MONDO:0007523"
condition_2 = df_term_association['object'].str.startswith("HP:")
df_100 = df_term_association[condition_1 & condition_2]
# df_100 = df_100[['subject', 'predicate', 'object']]
df_100

,id,subject,predicate,object,evidence_type,publication,source
120177,uuid:77611b21-42a9-11ee-be37-31ef105c25ea,MONDO:0007523,biolink:has_phenotype,HP:0100699,ECO:0000304,OMIM:130020,infores:hpo-annotations
120178,uuid:77611b22-42a9-11ee-be37-31ef105c25ea,MONDO:0007523,biolink:has_phenotype,HP:0001388,ECO:0000501,OMIM:130020,infores:hpo-annotations
120179,uuid:77611b23-42a9-11ee-be37-31ef105c25ea,MONDO:0007523,biolink:has_phenotype,HP:0001382,ECO:0000304,OMIM:130020,infores:hpo-annotations
120180,uuid:77611b24-42a9-11ee-be37-31ef105c25ea,MONDO:0007523,biolink:has_phenotype,HP:0001065,ECO:0000501,OMIM:130020,infores:hpo-annotations
120181,uuid:77611b25-42a9-11ee-be37-31ef105c25ea,MONDO:0007523,biolink:has_phenotype,HP:0001373,ECO:0000501,OMIM:130020,infores:hpo-annotations
...,...,...,...,...,...,...,...
149343,uuid:7914766d-42a9-11ee-be37-31ef105c25ea,MONDO:0007523,biolink:has_phenotype,HP:0012378,ECO:0000304,orphanet:285,infores:hpo-annotations
149344,uuid:7914766e-42a9-11ee-be37-31ef105c25ea,MONDO:0007523,biolink:has_phenotype,HP:0012732,ECO:0000304,orphanet:285,infores:hpo-annotations
149345,uuid:7914766f-42a9-11ee-be37-31ef105c25ea,MONDO:0007523,biolink:has_phenotype,HP:0100550,ECO:0000304,orphanet:285,infores:hpo-annotations
149346,uuid:79147670-42a9-11ee-be37-31ef105c25ea,MONDO:0007523,biolink:has_phenotype,HP:0100645,ECO:0000304,orphanet:285,infores:hpo-annotations


In [12]:
df_100['object'].drop_duplicates()

120177    HP:0100699
120178    HP:0001388
120179    HP:0001382
120180    HP:0001065
120181    HP:0001373
             ...    
149343    HP:0012378
149344    HP:0012732
149345    HP:0100550
149346    HP:0100645
149347    HP:0100823
Name: object, Length: 61, dtype: object

In [13]:
%%time
predicates = ["rdfs:subClassOf", "BFO:0000050"]
semsimian = Semsimian(
            spo=None,
            predicates=predicates,
            pairwise_similarity_attributes=None,
            resource_path=db,
        )


CPU times: user 15 s, sys: 2 s, total: 17 s
Wall time: 20.7 s


In [14]:

subject_prefixes = ["MGI:"]
object_terms = set(df_100['object'].drop_duplicates())
assoc_predicate = {"biolink:has_phenotype"}
include_similarity_object = True
limit = None


In [15]:
%%time

search_type = "full"

full_result = semsimian.associations_search(
            assoc_predicate,
            object_terms,
            include_similarity_object,
            search_type,
            None,
            subject_prefixes,
            limit,
        )

CPU times: user 59min 25s, sys: 1min 7s, total: 1h 32s
Wall time: 1h 1min


In [16]:
sorted(Counter(x[0] for x in full_result).items(), key=lambda x: x[0], reverse=True)

[(15.423914705247961, 26),
 (14.549445587331821, 28),
 (14.308437487828025, 239),
 (14.201522283911514, 5),
 (13.921414364718778, 15),
 (13.686949111081756, 6),
 (13.423914705247961, 1),
 (13.308437487828025, 41),
 (13.25398970380565, 26),
 (13.201522283911514, 66),
 (13.1019866103606, 78),
 (12.921414364718778, 8),
 (12.838952204526805, 29),
 (12.485315249912105, 42),
 (12.25398970380565, 5),
 (11.900352749190949, 197),
 (11.879594189024152, 10),
 (11.859130086464436, 19),
 (11.780058515473236, 4),
 (11.742090665274215, 6),
 (11.599486269831417, 36),
 (11.549445587331821, 35),
 (11.336451863997622, 186),
 (11.280956751405919, 309),
 (11.214461339619012, 7),
 (11.150896210841545, 25),
 (11.07813986840623, 26),
 (11.054680895582242, 7),
 (11.04309292130703, 106),
 (10.942788015511345, 21),
 (10.932061608918287, 16),
 (10.900352749190949, 742),
 (10.88993613324561, 8),
 (10.849005869190728, 32),
 (10.770472466644039, 25),
 (10.705096457792015, 15),
 (10.695994250684763, 562),
 (10.677960

In [17]:
full_result[0][2]

'MGI:104756'

In [18]:
full_result[0][1]['object_best_matches']

{'HP:0000023': {'match_source': 'HP:0000023',
  'match_source_label': 'Inguinal hernia (HPO)',
  'match_target': 'MP:0003660',
  'match_target_label': 'chylothorax (MPO)',
  'score': '2.697873983902985'},
 'HP:0000140': {'match_source': 'HP:0000140',
  'match_source_label': 'Abnormality of the menstrual cycle (HPO)',
  'match_target': 'MP:0003660',
  'match_target_label': 'chylothorax (MPO)',
  'score': '5.031955138118038'},
 'HP:0000144': {'match_source': 'HP:0000144',
  'match_source_label': 'Decreased fertility (HPO)',
  'match_target': 'MP:0003660',
  'match_target_label': 'chylothorax (MPO)',
  'score': '5.031955138118038'},
 'HP:0000164': {'match_source': 'HP:0000164',
  'match_source_label': 'Abnormality of the dentition (HPO)',
  'match_target': 'MP:0020855',
  'match_target_label': 'decreased corneal epithelium thickness (MPO)',
  'score': '5.061605553508769'},
 'HP:0000168': {'match_source': 'HP:0000168',
  'match_source_label': 'Abnormality of the gingiva (HPO)',
  'match_ta

In [19]:
%%time
search_type = "flat"

flat_result = semsimian.associations_search(
            assoc_predicate,
            object_terms,
            include_similarity_object,
            search_type,
            None,
            subject_prefixes,
            limit,
        )

CPU times: user 12.1 s, sys: 490 ms, total: 12.5 s
Wall time: 11.8 s


In [20]:
sorted(Counter(x[0] for x in flat_result).items(), key=lambda x: x[0], reverse=True)

[(0.11347517730496454, 1),
 (0.11181434599156118, 1),
 (0.10754189944134078, 1),
 (0.1046831955922865, 1),
 (0.1038107752956636, 1),
 (0.10334788937409024, 1),
 (0.10300429184549356, 1),
 (0.10277324632952692, 1),
 (0.10248901903367497, 1),
 (0.10243055555555555, 1),
 (0.10241820768136557, 1),
 (0.10220125786163523, 1),
 (0.10214168039538715, 1),
 (0.10209424083769633, 1),
 (0.10197368421052631, 1),
 (0.10196560196560196, 1),
 (0.10191846522781775, 1),
 (0.1017156862745098, 1),
 (0.1016949152542373, 1),
 (0.10139860139860139, 1),
 (0.10085227272727272, 1),
 (0.10076335877862595, 1),
 (0.09818181818181818, 1),
 (0.09813664596273292, 1),
 (0.09807073954983923, 1),
 (0.09792284866468842, 1),
 (0.09759547383309759, 1),
 (0.09746835443037975, 1),
 (0.09745762711864407, 1),
 (0.09739524348810873, 1),
 (0.09733124018838304, 1),
 (0.09673659673659674, 1),
 (0.09664948453608248, 1),
 (0.09656084656084656, 1),
 (0.09630606860158311, 1),
 (0.0962406015037594, 1),
 (0.09617612977983778, 1),
 (0.09

In [ ]:
%%time
search_type = "hybrid"

hybrid_result = semsimian.associations_search(
            assoc_predicate,
            object_terms,
            include_similarity_object,
            search_type,
            None,
            subject_prefixes,
            limit,
        )

In [ ]:
sorted(Counter(x[0] for x in hybrid_result).items(), key=lambda x: x[0], reverse=True)

In [ ]:
len(hybrid_result)

In [ ]:
full_result_score_curie = [(a,c) for (a, _, c) in full_result]
flat_result_score_curie = [(a,c) for (a, _, c) in flat_result]
hybrid_result_score_curie = [(a,c) for (a, _, c) in hybrid_result]

In [ ]:
len(full_result_score_curie)

In [ ]:
len(flat_result_score_curie)

In [ ]:
def get_ordered_list(result_1, result_2):
    # Create a dictionary that maps MGI values to tuples in result_2
    result_2_dict = {mgi: tup for tup, mgi in result_2}

    # Create a new list of tuples from result_2 that match the MGI values in result_1
    result_2_pruned = [(result_2_dict[mgi], mgi) for _, mgi in result_1 if mgi in result_2_dict]
    return result_2_pruned

### Plot params

In [ ]:
n = 50
full_result_score_curie_subset = full_result_score_curie[:n]
full_ordered_flat_subset = get_ordered_list(full_result_score_curie_subset, flat_result_score_curie)
full_ordered_hybrid_subset = get_ordered_list(full_result_score_curie_subset, hybrid_result_score_curie)

# print(full_result_score_curie_subset)
# print(full_ordered_flat_subset)
# print(full_ordered_hybrid_subset)

In [ ]:
flat_result_score_curie_subset = flat_result_score_curie[:n]
flat_ordered_full_subset = get_ordered_list(flat_result_score_curie_subset, full_result_score_curie)
flat_ordered_hybrid_subset = get_ordered_list(flat_result_score_curie_subset, hybrid_result_score_curie)

# print(flat_result_score_curie_subset)
# print(flat_ordered_full_subset)
# print(flat_ordered_hybrid_subset)


In [ ]:
hybrid_result_score_curie_subset = hybrid_result_score_curie[:n]
hybrid_ordered_full_subset = get_ordered_list(hybrid_result_score_curie_subset, full_result_score_curie)
hybrid_ordered_flat_subset = get_ordered_list(hybrid_result_score_curie_subset, flat_result_score_curie)

# print(hybrid_result_score_curie_subset)
# print(hybrid_ordered_full_subset)
# print(hybrid_ordered_flat_subset)


In [ ]:
import matplotlib.pyplot as plt
import numpy as np
from adjustText import adjust_text


def plot_graph(full_result, flat_result, hybrid_result, title):
    full_values, full_labels = zip(*full_result)
    flat_values, flat_labels = zip(*flat_result)
    hybrid_values, hybrid_labels = zip(*hybrid_result)

    # Create a figure and a set of subplots
    fig, ax = plt.subplots(figsize=(10, 6))

    # Add a title
    if title:
        ax.set_title(title)

    # # Convert labels to numerical values
    # unique_labels = list(set(full_labels + flat_labels + hybrid_labels))
    # label_to_num = {label: num for num, label in enumerate(unique_labels)}

    # full_labels = [label_to_num[label] for label in full_labels]
    # flat_labels = [label_to_num[label] for label in flat_labels]
    # hybrid_labels = [label_to_num[label] for label in hybrid_labels]

    # Plot 'full' with 'o' markers
    ax.scatter(full_labels, full_values, marker='o', color='r', label='full search')
    texts1 = [plt.text(full_labels[i], full_values[i], str(txt), rotation=90) for i, txt in enumerate(full_values)]
    
    # Plot 'flat' with 'x' markers
    ax.scatter(flat_labels, flat_values, marker='x', color='b', label='flat search')
    texts2 = [plt.text(flat_labels[i], flat_values[i], str(txt), rotation=90) for i, txt in enumerate(flat_values)]
    
    # Plot 'hybrid' with 'x' markers
    ax.scatter(hybrid_labels, hybrid_values, marker='*', color='g', label='hybrid search')
    texts3 = [plt.text(hybrid_labels[i], hybrid_values[i], str(txt), rotation=90) for i, txt in enumerate(hybrid_values)]
    
    
    # Add a legend
    ax.legend()
    
    # Rotate x-axis labels
    plt.xticks(rotation=90)
    
    # all_texts = texts1+texts2+texts3
    
    # # Adjust text to avoid overlaps
    # adjust_text(all_texts, autoalign='y', only_move={'points':'y', 'text':'y'})
    
    # Show the plot
    plt.show()

In [ ]:
print("Ordered by Full search ranks")
plot_graph(full_result_score_curie_subset, full_ordered_flat_subset, full_ordered_hybrid_subset, None)

In [ ]:
print( "Ordered by Flat search ranks")

plot_graph(flat_ordered_full_subset, flat_result_score_curie_subset, flat_ordered_hybrid_subset, None)

In [ ]:

print("Ordered by Hybrid search ranks")
plot_graph(hybrid_ordered_full_subset, hybrid_ordered_flat_subset, hybrid_result_score_curie_subset, None)